In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import utilsforecast.losses as ufl

In [3]:
from utilsforecast.data import generate_series

In [4]:
series = generate_series(1_000, n_models=2)
series.head()

,unique_id,ds,y,model0,model1
0,0,2000-01-01,0.495571,0.483500,0.496926
1,0,2000-01-02,1.191551,1.106211,1.121050
2,0,2000-01-03,2.403335,2.562420,2.431159
3,0,2000-01-04,3.230503,3.127327,2.979246
4,0,2000-01-05,4.413412,4.817242,4.612489


In [5]:
models=['model0', 'model1']

In [6]:
%time r1 = ufl.mse(series, models=models)

CPU times: user 16.2 ms, sys: 3.6 ms, total: 19.8 ms
Wall time: 21.1 ms


In [8]:
import narwhals as nw
from narwhals.typing import IntoDataFrameT

In [9]:
nw.from_native(series)

┌─────────────────────────────────────────────────────────┐
|                   Narwhals DataFrame                    |
|---------------------------------------------------------|
|       unique_id         ds         y    model0    model1|
|0              0 2000-01-01  0.495571  0.483500  0.496926|
|1              0 2000-01-02  1.191551  1.106211  1.121050|
|2              0 2000-01-03  2.403335  2.562420  2.431159|
|3              0 2000-01-04  3.230503  3.127327  2.979246|
|4              0 2000-01-05  4.413412  4.817242  4.612489|
|...          ...        ...       ...       ...       ...|
|277276       999 2000-12-31  6.067813  5.761869  5.690605|
|277277       999 2001-01-01  0.088384  0.094595  0.086874|
|277278       999 2001-01-02  1.121854  1.228645  1.231418|
|277279       999 2001-01-03  2.442165  2.569426  2.348864|
|277280       999 2001-01-04  3.144067  3.458452  3.440719|
|                                                         |
|[277281 rows x 5 columns]              

In [159]:
def nw_agg(df, models, id_col, gen_expr, agg='mean'):
    df = nw.from_native(df)
    exprs = [gen_expr(m) for m in models]
    return (
        df.select([id_col, *exprs])
        .group_by(id_col)
        .agg(getattr(nw.col(models), agg)())
        .to_native()
    )

def nw_mse(
    df: IntoDataFrameT,
    models: list[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> IntoDataFrameT:
    return nw_agg(
        df,
        models,
        id_col,
        lambda m: ((nw.col(target_col) - nw.col(m)) ** 2).alias(m)
    )

def nw_mae(
    df: IntoDataFrameT,
    models: list[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> IntoDataFrameT:
    return nw_agg(
        df,
        models,
        id_col,
        lambda m: (nw.col(target_col) - nw.col(m)).abs().alias(m)
    )

In [15]:
ufl.mae(series, models)

,unique_id,model0,model1
0,0,0.145806,0.165170
1,1,0.184338,0.165456
2,2,0.161968,0.146885
3,3,0.175651,0.159350
4,4,0.158874,0.163099
...,...,...,...
995,995,0.174550,0.172263
996,996,0.161186,0.157935
997,997,0.159059,0.159964
998,998,0.166432,0.180902


In [16]:
ufl.bias(series, models)

,unique_id,model0,model1
0,0,-0.014454,-0.004884
1,1,0.010106,-0.005386
2,2,-0.002340,-0.017991
3,3,-0.024564,-0.016486
4,4,0.000497,0.001391
...,...,...,...
995,995,-0.068328,-0.041908
996,996,0.000997,0.004755
997,997,0.003365,-0.004519
998,998,0.021729,-0.006692


In [17]:
ufl.cfe(series, models)

,unique_id,model0,model1
0,0,-3.208692,-1.084266
1,1,0.980279,-0.522455
2,2,-0.390744,-3.004500
3,3,-5.944555,-3.989711
4,4,0.185344,0.518674
...,...,...,...
995,995,-7.652764,-4.693710
996,996,0.442703,2.111377
997,997,1.500703,-2.015648
998,998,3.107293,-0.956910


In [19]:
ufl.pis(series, models)

,unique_id,model0,model1
0,0,32.368852,36.667816
1,1,17.880831,16.049236
2,2,27.048582,24.529851
3,3,42.507456,38.562751
4,4,59.260082,60.835792
...,...,...,...
995,995,19.549628,19.293410
996,996,71.566648,70.122958
997,997,70.940406,71.344048
998,998,23.799786,25.868944


In [27]:
y = nw.col("y")
nw.from_native(series).with_columns(
    a=nw.when(y == 0).then(float("nan")).otherwise(y),
    #b=y.replace_strict({0: float("nan")})
)

┌───────────────────────────────────────────────────────────────────┐
|                        Narwhals DataFrame                         |
|-------------------------------------------------------------------|
|       unique_id         ds         y    model0    model1         a|
|0              0 2000-01-01  0.495571  0.483500  0.496926  0.495571|
|1              0 2000-01-02  1.191551  1.106211  1.121050  1.191551|
|2              0 2000-01-03  2.403335  2.562420  2.431159  2.403335|
|3              0 2000-01-04  3.230503  3.127327  2.979246  3.230503|
|4              0 2000-01-05  4.413412  4.817242  4.612489  4.413412|
|...          ...        ...       ...       ...       ...       ...|
|277276       999 2000-12-31  6.067813  5.761869  5.690605  6.067813|
|277277       999 2001-01-01  0.088384  0.094595  0.086874  0.088384|
|277278       999 2001-01-02  1.121854  1.228645  1.231418  1.121854|
|277279       999 2001-01-03  2.442165  2.569426  2.348864  2.442165|
|277280       999 20

In [47]:
series2 = series.copy()
series2.loc[series2['unique_id'] == 0, 'y'] = 0 

In [48]:
ufl.mape(series2, models)

,unique_id,model0,model1
0,0,NaN,NaN
1,1,0.055817,0.051974
2,2,0.050458,0.046900
3,3,0.052462,0.048354
4,4,0.049400,0.050532
...,...,...,...
995,995,0.052832,0.050262
996,996,0.049165,0.049361
997,997,0.050028,0.048472
998,998,0.051738,0.053235


In [50]:
ufl.spis(series, series, models)

,unique_id,model0,model1
0,0,10.025300,11.356777
1,1,5.536832,4.969675
2,2,8.301312,7.528303
3,3,12.983744,11.778849
4,4,18.320591,18.807731
...,...,...,...
995,995,5.979635,5.901266
996,996,22.059643,21.614641
997,997,21.718674,21.842250
998,998,7.322024,7.958602


In [58]:
target_col = 'y'
seasonality = 7
id_col = 'unique_id'

def my_expr(m):
    lagged = nw.col(m).shift(seasonality).over(id_col)
    return (nw.col(target_col) - lagged).abs().alias("scale")

ufl._nw_agg_expr(
    series,
    models=['y'],
    id_col='unique_id',
    gen_expr=my_expr,
)

/Users/jose.morales/miniforge3/envs/utils/lib/python3.11/site-packages/narwhals/_pandas_like/expr.py:282: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df._native_frame.groupby(partition_by)


,unique_id,scale
0,0,0.178931
1,1,0.179144
2,2,0.180418
3,3,0.176610
4,4,0.171043
...,...,...
995,995,0.176833
996,996,0.167311
997,997,0.171825
998,998,0.194124


In [49]:
ufl.smape(series, models)

,unique_id,model0,model1
0,0,0.023896,0.025068
1,1,0.027919,0.026043
2,2,0.025293,0.023540
3,3,0.026364,0.024243
4,4,0.024744,0.025341
...,...,...,...
995,995,0.026748,0.025347
996,996,0.024622,0.024675
997,997,0.025006,0.024267
998,998,0.025819,0.026679


In [124]:
ufl.msse(series, models, 7, series)

/Users/jose.morales/miniforge3/envs/utils/lib/python3.11/site-packages/narwhals/_pandas_like/expr.py:282: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df._native_frame.groupby(partition_by)


,unique_id,model0,model1
0,0,0.836489,1.084589
1,1,1.256558,1.063925
2,2,1.042767,0.875167
3,3,1.182429,1.002119
4,4,1.039306,1.130435
...,...,...,...
995,995,1.206832,1.284414
996,996,1.162545,1.079721
997,997,1.064778,1.114471
998,998,0.989667,1.148005


In [125]:
ufl.rmsse(series, models, 7, series)

/Users/jose.morales/miniforge3/envs/utils/lib/python3.11/site-packages/narwhals/_pandas_like/expr.py:282: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df._native_frame.groupby(partition_by)


,unique_id,model0,model1
0,0,0.914598,1.041436
1,1,1.120963,1.031468
2,2,1.021160,0.935504
3,3,1.087396,1.001059
4,4,1.019463,1.063219
...,...,...,...
995,995,1.098559,1.133320
996,996,1.078214,1.039096
997,997,1.031881,1.055685
998,998,0.994820,1.071450


In [137]:
series = generate_series(1_000, n_models=2, level=[50])

In [138]:
series

,unique_id,ds,y,model0,model0-lo-50,model0-hi-50,model1,model1-lo-50,model1-hi-50
0,0,2000-01-01,0.495571,0.483500,0.437784,0.529216,0.496926,0.433112,0.560740
1,0,2000-01-02,1.191551,1.106211,1.066970,1.145451,1.121050,1.063831,1.178269
2,0,2000-01-03,2.403335,2.562420,2.030098,3.094743,2.431159,2.092081,2.770237
3,0,2000-01-04,3.230503,3.127327,2.861262,3.393392,2.979246,2.896485,3.062006
4,0,2000-01-05,4.413412,4.817242,3.664111,5.970372,4.612489,3.775857,5.449120
...,...,...,...,...,...,...,...,...,...
277276,999,2000-12-31,6.067813,5.761869,5.404783,6.118955,5.690605,5.421478,5.959733
277277,999,2001-01-01,0.088384,0.094595,0.074462,0.114728,0.086874,0.077870,0.095878
277278,999,2001-01-02,1.121854,1.228645,0.928869,1.528420,1.231418,0.927160,1.535675
277279,999,2001-01-03,2.442165,2.569426,2.080882,3.057971,2.348864,2.167426,2.530301


In [153]:
ufl.quantile_loss(series, models={'model0': 'model0-lo-50', 'model1': 'model1-lo-50'})

,unique_id,model0,model1
0,0,0.204784,0.207919
1,1,0.211867,0.207964
2,2,0.210034,0.205891
3,3,0.207572,0.207963
4,4,0.208776,0.209343
...,...,...,...
995,995,0.199120,0.204151
996,996,0.209809,0.210128
997,997,0.211400,0.209859
998,998,0.214320,0.209836


In [152]:
ufl.quantile_loss(series, models={'model0': 'model0-lo-50', 'model1': 'model1-lo-50'})

,unique_id,model0,model1
0,0,0.204784,0.207919
1,1,0.211867,0.207964
2,2,0.210034,0.205891
3,3,0.207572,0.207963
4,4,0.208776,0.209343
...,...,...,...
995,995,0.199120,0.204151
996,996,0.209809,0.210128
997,997,0.211400,0.209859
998,998,0.214320,0.209836


In [162]:
import numpy as np

In [163]:
ufl.scaled_quantile_loss(series, models={'model0': 'model0-lo-50', 'model1': 'model1-lo-50'}, seasonality=7, train_df=series)

/Users/jose.morales/miniforge3/envs/utils/lib/python3.11/site-packages/narwhals/_pandas_like/expr.py:282: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df._native_frame.groupby(partition_by)


,unique_id,model0,model1
0,0,1.144481,1.162004
1,1,1.182665,1.160877
2,2,1.164152,1.141191
3,3,1.175317,1.177527
4,4,1.220606,1.223919
...,...,...,...
995,995,1.126030,1.154484
996,996,1.254005,1.255911
997,997,1.230322,1.221354
998,998,1.104037,1.080938


In [187]:
ufl.mqloss(series, models={'model0': ['model0-lo-50'], 'model1': ['model1-lo-50']}, quantiles=np.array([0.5]))

,unique_id,model0,model1
0,0,0.204784,0.207919
1,1,0.211867,0.207964
2,2,0.210034,0.205891
3,3,0.207572,0.207963
4,4,0.208776,0.209343
...,...,...,...
995,995,0.199120,0.204151
996,996,0.209809,0.210128
997,997,0.211400,0.209859
998,998,0.214320,0.209836


In [189]:
series = generate_series(1_000, n_models=2, level=[80, 95])

In [190]:
ufl.coverage(series, ['model0'], 80)

,unique_id,model0
0,0,0.819820
1,1,0.773196
2,2,0.820359
3,3,0.772727
4,4,0.828418
...,...,...
995,995,0.714286
996,996,0.824324
997,997,0.849776
998,998,0.839161


In [196]:
ufl.calibration(series, models={'model0': ['model0-hi-80']})

,unique_id,model0
0,0,0.819820
1,1,0.773196
2,2,0.820359
3,3,0.772727
4,4,0.828418
...,...,...
995,995,0.714286
996,996,0.824324
997,997,0.849776
998,998,0.839161


In [232]:
import pandas as pd
pd.DataFrame([
    {'x': 1, 'y': 2, 'z': 3},
    {'x': 2, 'y': 3, 'z': 4},
])

,x,y,z
0,1,2,3
1,2,3,4


In [229]:
next(iter(series.groupby('unique_id')))

/var/folders/q3/2nf6vtsj52xgby1cr8qn_vgr0000gp/T/ipykernel_57308/2465714687.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  next(iter(series.groupby('unique_id')))


(0,
     unique_id         ds         y    model0  model0-lo-80  model0-hi-80  \
 0           0 2000-01-01  0.495571  0.483500      0.410354      0.556646   
 1           0 2000-01-02  1.191551  1.106211      1.043426      1.168996   
 2           0 2000-01-03  2.403335  2.562420      1.710704      3.414136   
 3           0 2000-01-04  3.230503  3.127327      2.701624      3.553031   
 4           0 2000-01-05  4.413412  4.817242      2.972233      6.662250   
 ..        ...        ...       ...       ...           ...           ...   
 217         0 2000-08-05  0.342739  0.350911      0.263995      0.437827   
 218         0 2000-08-06  1.115535  1.140716      0.861074      1.420357   
 219         0 2000-08-07  2.269434  2.077145      2.013709      2.140581   
 220         0 2000-08-08  3.386517  3.496069      2.570665      4.421472   
 221         0 2000-08-09  4.373292  4.162012      3.731755      4.592268   
 
      model0-lo-95  model0-hi-95    model1  model1-lo-80  model1-hi-80

In [228]:
nwdf

┌─────────────────────────────────────────────────────────┐
|                   Narwhals DataFrame                    |
|---------------------------------------------------------|
|       unique_id         ds         y    model0    model1|
|0              0 2000-01-01  0.495571  0.483500  0.496926|
|1              0 2000-01-02  1.191551  1.106211  1.121050|
|2              0 2000-01-03  2.403335  2.562420  2.431159|
|3              0 2000-01-04  3.230503  3.127327  2.979246|
|4              0 2000-01-05  4.413412  4.817242  4.612489|
|...          ...        ...       ...       ...       ...|
|277276       999 2000-12-31  6.067813  5.761869  5.690605|
|277277       999 2001-01-01  0.088384  0.094595  0.086874|
|277278       999 2001-01-02  1.121854  1.228645  1.231418|
|277279       999 2001-01-03  2.442165  2.569426  2.348864|
|277280       999 2001-01-04  3.144067  3.458452  3.440719|
|                                                         |
|[277281 rows x 5 columns]              

In [227]:
ufl.tweedie_deviance(series, models, power=0)

,unique_id,model0,model1
0,0,0.039292,0.050946
1,1,0.059355,0.050256
2,2,0.048132,0.040396
3,3,0.054658,0.046323
4,4,0.045806,0.049822
...,...,...,...
995,995,0.054620,0.058131
996,996,0.049030,0.045537
997,997,0.045923,0.048066
998,998,0.051068,0.059239


In [214]:
ufl.scaled_crps(series, models={'model0': ['model0-hi-80']}, quantiles=[0.2, 0.3])

,unique_id,model0
0,0,0.160234
1,1,0.184144
2,2,0.171701
3,3,0.159158
4,4,0.173927
...,...,...
995,995,0.127409
996,996,0.174954
997,997,0.176762
998,998,0.190944


In [197]:
import utilsforecast.processing as ufp

In [198]:
ufp.counts_by_id??

Signature:
ufp.counts_by_id(
    df: Union[pandas.core.frame.DataFrame, polars.dataframe.frame.DataFrame],
    id_col: str,
) -> Union[pandas.core.frame.DataFrame, polars.dataframe.frame.DataFrame]
Docstring: <no docstring>
Source:   
def counts_by_id(df: DataFrame, id_col: str) -> DataFrame:
    if isinstance(df, pd.DataFrame):
        id_counts = df[id_col].value_counts(sort=False, dropna=True)
        ids = id_counts.index
        if isinstance(ids.dtype, pd.CategoricalDtype):
            # there's no observed argument in value_counts
            # so this can return unseen categories
            id_counts = id_counts[id_counts > 0]
            ids = id_counts.index.codes
        sort_idxs = ids.argsort()
        id_counts = id_counts.iloc[sort_idxs].reset_index()
    else:
        id_counts = df[id_col].value_counts().sort(id_col)
    id_counts.columns = [id_col, "counts"]
    return id_counts
File:      ~/dev/repos/utilsforecast/utilsforecast/processing.py
Type:      function

In [165]:
%time r2 = nw_mae(series, models)

CPU times: user 14.2 ms, sys: 1.98 ms, total: 16.2 ms
Wall time: 15 ms


In [105]:
nw.from_native(series)['y'].is_null().any()

np.False_

In [107]:
nwdf = nw.from_native(series)

In [113]:
nwdf.select(nw.col("y") / nw.col("y").sum().over("unique_id"))

/Users/jose.morales/miniforge3/envs/utils/lib/python3.11/site-packages/narwhals/_pandas_like/expr.py:282: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df._native_frame.groupby(partition_by)


┌─────────────────────────┐
|   Narwhals DataFrame    |
|-------------------------|
|               y         |
|0       0.000691         |
|1       0.001662         |
|2       0.003353         |
|3       0.004507         |
|4       0.006157         |
|...          ...         |
|277276  0.005049         |
|277277  0.000074         |
|277278  0.000934         |
|277279  0.002032         |
|277280  0.002616         |
|                         |
|[277281 rows x 1 columns]|
└─────────────────────────┘

In [153]:
pd.testing.assert_frame_equal(r1, r2)

In [147]:
%timeit nw_mse(series, models)

8.97 ms ± 228 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [119]:
nwdf.with_columns(lagged=nw.col("y").shift(1).over("unique_id"))

/Users/jose.morales/miniforge3/envs/utils/lib/python3.11/site-packages/narwhals/_pandas_like/expr.py:282: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df._native_frame.groupby(partition_by)


┌───────────────────────────────────────────────────────────────────┐
|                        Narwhals DataFrame                         |
|-------------------------------------------------------------------|
|       unique_id         ds         y    model0    model1    lagged|
|0              0 2000-01-01  0.495571  0.483500  0.496926       NaN|
|1              0 2000-01-02  1.191551  1.106211  1.121050  0.495571|
|2              0 2000-01-03  2.403335  2.562420  2.431159  1.191551|
|3              0 2000-01-04  3.230503  3.127327  2.979246  2.403335|
|4              0 2000-01-05  4.413412  4.817242  4.612489  3.230503|
|...          ...        ...       ...       ...       ...       ...|
|277276       999 2000-12-31  6.067813  5.761869  5.690605  5.316534|
|277277       999 2001-01-01  0.088384  0.094595  0.086874  6.067813|
|277278       999 2001-01-02  1.121854  1.228645  1.231418  0.088384|
|277279       999 2001-01-03  2.442165  2.569426  2.348864  1.121854|
|277280       999 20

In [117]:
ufl.nd(series, models)

,unique_id,model0,model1
0,0,0.047611,0.050069
1,1,0.055817,0.051974
2,2,0.050458,0.046900
3,3,0.052462,0.048354
4,4,0.049400,0.050532
...,...,...,...
995,995,0.052832,0.050262
996,996,0.049165,0.049361
997,997,0.050028,0.048472
998,998,0.051738,0.053235


In [120]:
ufl.msse(series, models, 7, series)

/Users/jose.morales/miniforge3/envs/utils/lib/python3.11/site-packages/narwhals/_pandas_like/expr.py:282: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df._native_frame.groupby(partition_by)


TypeError: _scale_loss() got an unexpected keyword argument 'target_col'

In [146]:
r2 = nw_mse(series, models)

In [27]:
import pandas as pd

In [109]:
pd.testing.assert_frame_equal(r1, r2)

In [97]:
import polars as pl

In [110]:
series_pl = generate_series(1_000, n_models=2, engine='polars')

In [118]:
%timeit ufl.mse(series_pl, models=models)

1.22 ms ± 12.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [119]:
%timeit nw_mse(series_pl, models)

1.36 ms ± 16.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [111]:
%time r1_pl = ufl.mse(series_pl, models=models)

CPU times: user 9.79 ms, sys: 12.3 ms, total: 22 ms
Wall time: 3.82 ms


In [117]:
%time r2_pl = nw_mse(series_pl, models)

CPU times: user 9.24 ms, sys: 8.42 ms, total: 17.7 ms
Wall time: 3.1 ms


In [135]:
%time r2_pl = nw_mse(series_pl, models)

CPU times: user 10.8 ms, sys: 8.87 ms, total: 19.6 ms
Wall time: 3.86 ms
